# Pairwise DNA/Protein Alignment

## Import Libraries

In [ ]:
#import libraries

import numpy as np
import pandas as pd

## Read BLOSUM and DNA score Matrices

In [ ]:
#The following codes read the csv files of substitution matrices
#Prior to running the codes, please upload the corresponding csv files.

blosum_50_df = pd.read_csv('blosum_50.csv', index_col = [0])
blosum_62_df = pd.read_csv('blosum_62.csv', index_col = [0])
blosum_80_df = pd.read_csv('blosum_80.csv', index_col = [0])

dna_sub_matrix = pd.read_csv('dna_sub_matrix.csv', index_col = [0])

## Composite Functions of Aligntwo

### The __Aligntwo()__ function is a user defined  function that consists of 5 separate smaller functions. Here are the list of functions used to create Aligntwo

1. __score_func(__xi, yj, go, ge, gap_opening, sub_matrix __)__: returns the similarity score of two amino acid residues or bases. 

  arguments: 

  * _xi_: first amino acid residue/base
  * _yj_: second amino acid residue/base
  * _go_: gap opening penalty. Default penalty is -2
  * _ge_: gap extension penalty. Default penalty is -2
  * _gap_opening_: True/False. True if the preceding alignment is a gap and false if otherwise. 

2. __get_S(__ seq1, seq2, sub_matrix = blosum_62_df, go = -2, ge = -2, align_type = 'gbl' __)__ : 
get_S function returns a Score Matrix of two protein/DNA sequences. 

  arguments:

  * _seq1_: first protein/DNA sequence
  * _seq2_: second protein/DNA sequence
  * _sub_matrix_: substitution matrix. Default substitution matrix is BLOSUM62. If the user wishes to use different substitution matrix, enter the dataframe. (e.g. sub_matrix = blosum_80_df, sub_matrix = dna_sub_matrix)
  * _go_: gap opening penalty. Default penalty is -2
  * _ge_: gap extension penalty. Default penalty is -2
  * _align_type_ : type of alignment. Either 'gbl'(global alignment) or 'lcl'(local alignment). Default alignment type is 'gbl'

3. __find_optim_path(__S, position_dir, nrow, ncol, path_dir = [], align_type = 'gbl' __)__ 

  :find_optim_path function returns a list of directions ('right', 'down', 'diagonal') of the optimal path from the upper left corner to the lower right corner of the score matrix. 

  arguments:

  * _S_: score matrix
  * _position_dir_: dataframe containing the directions of the substitution matrix. Values are either 'right','down' or 'diagonal'
  * _path_dir_: list that will store the directions of the optimal path from the upper left corner to the lower right corner of the score matrix. 
  * _align_type_: alignment type. If align_type = 'gbl', function returns path_dir. Otherwise, it will return None. 

4. __visualize_global_alignment(__ seq1, seq2, path_dir __)__ 
  : visualize_global_alignment is a function which visualizes the optimal global alignment of two sequences. 

  arguments:

  * _seq1_: first Protein/DNA sequence
  * _seq2_: second Protein/DNA sequence
  * _path_dir_: list containing the directions of the optimal path from the upper left corner to the lower right corner of the score matrix. 

5. __visualize_local_alignment(__ seq1, seq2, S, position_dir __)__ 
  : visualize_local_alignment is a function which visualizes the optimal local alignment of two sequences. 

  arguments: 

  * _seq1_ : first Protein/DNA sequence
  * _seq2_ : second Protein/DNA sequence
  * _S_: score matrix
  * _position_dir_: dataframe containing the directions in the substitution matrix



In [ ]:
def score_func(xi, yj, go = -2, ge = -2, gap_opening = False, sub_matrix = blosum_62_df):
    if gap_opening == False:
        if xi == '-' or yj == '-':
            return go #gap opening penalty
        else:
            return sub_matrix.loc[xi, yj]
        
    elif gap_opening == True:
        if xi == '-' or yj == '-':
            return ge #gap extension penalty
        else:
            return sub_matrix.loc[xi, yj]

In [ ]:
def get_S(seq1, seq2, sub_matrix = blosum_62_df, go = -2, ge = -2, align_type = 'gbl'):
    
    M = len(seq1)
    N = len(seq2)
    
    S = np.empty(shape=(M + 1, N + 1), dtype='object') #substitution matrix
    position_dir = pd.DataFrame(index = [i for i in range(len(seq1))], 
                            columns = [j for j in range(len(seq2))])

    for i in range(M + 1):
        S[i,0] = 0

    for j in range(N + 1):
        S[0,j] = 0

    right_gap_opening = False
    down_gap_opening = False

    for i in range(1, M+1):
        for j in range(1, N+1):

            xi = seq1[i-1]
            yj = seq2[j-1]

            if i >= 2:
                if position_dir.loc[i-2,j-1] == 'down':
                    down_gap_opening = True
                    
                elif position_dir.loc[i-2,j-1] != 'down':
                    down_gap_opening = False

            if j >= 2:
                if  position_dir.loc[i-1,j-2] == 'right':
                    right_gap_opening = True

                elif position_dir.loc[i-1,j-2] != 'right':
                    right_gap_opening = False


            right = S[i,j-1] + score_func(xi, '-', go = go, ge = ge,
                                          gap_opening = right_gap_opening, 
                                          sub_matrix = sub_matrix)
            down = S[i-1,j] + score_func('-', yj, go = go, ge = ge, 
                                         gap_opening = down_gap_opening, 
                                         sub_matrix = sub_matrix)
            diagonal = S[i-1, j-1] + score_func(xi, yj, go = go, ge = ge,
                                                gap_opening = False, 
                                                sub_matrix = sub_matrix)

            S[i, j] = np.max([right, down, diagonal])

            if S[i,j] == diagonal:
                position_dir.loc[i-1,j-1] = 'diagonal'
            elif S[i,j] == right:
                position_dir.loc[i-1,j-1] = 'right'
            elif S[i,j] == down:
                position_dir.loc[i-1,j-1] = 'down'
                
    if align_type == 'gbl':
        return S, position_dir
    elif align_type == 'lcl':
        return S * (S > 0), position_dir

In [ ]:
def find_optim_path(S, position_dir, nrow, ncol, path_dir = [], align_type = 'gbl'):
    
    if align_type == 'gbl':
        
        if nrow == 0 and ncol == 0:
            path_dir.reverse()
            return path_dir

        elif nrow != 0 and ncol != 0:

            #print("nrow: {}, ncol: {}, path_dir: {}".format(nrow, ncol, path_dir))
            direction = position_dir.iloc[nrow - 1, ncol - 1]

            if direction == 'diagonal':
                path_dir.append('diagonal')
                return find_optim_path(S, position_dir, nrow - 1, ncol - 1, path_dir)
            if direction == 'right':
                path_dir.append('right')
                return find_optim_path(S, position_dir, nrow, ncol - 1, path_dir)
            if direction == 'down':
                path_dir.append('down')
                return find_optim_path(S, position_dir, nrow - 1, ncol, path_dir)

        elif nrow == 0:
            #print("nrow: {}, ncol: {}, path_dir: {}".format(nrow, ncol, path_dir))
            path_dir.append('right')
            return find_optim_path(S, position_dir, nrow, ncol - 1, path_dir)

        elif ncol == 0:
            #print("nrow: {}, ncol: {}, path_dir: {}".format(nrow, ncol, path_dir))
            path_dir.append('down')
            return find_optim_path(S, position_dir, nrow - 1, ncol, path_dir)
        
    elif align_type == 'lcl':
        
        return None

In [ ]:
def visualize_global_alignment(seq1, seq2, path_dir):
    aligned_seq1 = ''
    aligned_seq2 = ''
    match_mismatch = ''

    count1 = 0
    count2 = 0

    for direction in path_dir:
        if direction == 'down':
            aligned_seq2 += '_'
            aligned_seq1 += seq1[count1]
            match_mismatch += ' '
            count1 += 1
        elif direction == 'right':
            aligned_seq2 += seq2[count2]
            aligned_seq1 += '_'
            match_mismatch += ' '
            count2 += 1
        elif direction == 'mismatch':       
            aligned_seq2 += seq2[count2]
            aligned_seq1 += seq1[count1]
            count1 += 1
            count2 += 1
            match_mismatch += '*'
        elif direction == 'match':
            aligned_seq2 += seq2[count2]
            aligned_seq1 += seq1[count1]
            count1 += 1
            count2 += 1
            match_mismatch += '|' 
        elif direction == 'diagonal':
            aligned_seq2 += seq2[count2]
            aligned_seq1 += seq1[count1]
            
            if seq2[count2] == seq1[count1]:
                match_mismatch += '|'
            else:
                match_mismatch += '*'  
                
            count1 += 1
            count2 += 1
    
    aligned_seq_len = len(aligned_seq1)
    count = 0
            
    while aligned_seq_len > 50:
        print(aligned_seq1[50*count:50*(count+1)])
        print(match_mismatch[50*count:50*(count+1)])
        print(aligned_seq2[50*count:50*(count+1)])
        aligned_seq_len -= 50
        count += 1
    print(aligned_seq1[50*count:50*(count+1)])
    print(match_mismatch[50*count:50*(count+1)])
    print(aligned_seq2[50*count:50*(count+1)])

In [ ]:
def visualize_local_alignment(seq1, seq2, S, position_dir):

    max_value_index = list(zip(*np.where(S == np.max(S))))
    index_sum = np.sum(max_value_index, axis = 1)
    max_value_index = max_value_index[int(np.where(np.max(index_sum) == index_sum)[0])]

    score = S[max_value_index]
    index = max_value_index
    nrow = max_value_index[0]
    ncol = max_value_index[1]

    path_coord = []
    path_dir = []

    while score != 0:
        path_coord.append([nrow, ncol])
        direction = position_dir.iloc[nrow - 1, ncol - 1]
        path_dir.append(direction)
        #print('score:{}, coord:{},{}, direction:{}'.format(score, nrow, ncol, direction))
        if direction == 'diagonal':
            nrow -= 1
            ncol -= 1
        elif direction == 'right':
            ncol -= 1
        elif direction == 'down':
            nrow -= 1
        score = S[nrow, ncol]

    path_coord = np.array(path_coord)

    seq1_start = path_coord[:,0][-1]
    seq1_stop = path_coord[:,0][0]
    seq2_start = path_coord[:,1][-1]
    seq2_stop = path_coord[:,1][0]

    aligned_seq1 = ''
    aligned_seq2 = ''
    match_mismatch = ''
    
    nrow = seq1_start
    ncol = seq2_start
    
    path_dir.reverse()
    for direction in path_dir:
        if direction == 'diagonal':
            aligned_seq1 += seq1[nrow - 1]
            aligned_seq2 += seq2[ncol - 1]
            if seq1[nrow - 1] == seq2[ncol - 1]:
                match_mismatch += '|'
            else:
                match_mismatch += '*'
            nrow += 1
            ncol += 1
        elif direction == 'right':
            aligned_seq1 += '_'
            aligned_seq2 += seq2[ncol - 1]
            match_mismatch += ' '
            ncol += 1
        elif direction == 'down':
            aligned_seq1 += seq1[nrow - 1]
            aligned_seq2 += '_'
            match_mismatch += ' '
            nrow += 1
        #print(aligned_seq1, aligned_seq2)
    aligned_seq_len = len(aligned_seq1)
    count = 0
    while aligned_seq_len > 50:
        print(aligned_seq1[50*count:50*(count+1)])
        print(match_mismatch[50*count:50*(count+1)])
        print(aligned_seq2[50*count:50*(count+1)])
        aligned_seq_len -= 50
        count += 1
    print(aligned_seq1[50*count:50*(count+1)])
    print(match_mismatch[50*count:50*(count+1)])
    print(aligned_seq2[50*count:50*(count+1)])

## Aligntwo Function

    Aligntwo(seq1, seq2, sub_matrix = blosum_62_df, go = -2, ge = -2, align_type = 'gbl')

The Aligntwo function is a user defined function which visualizes the best global/local alignment of two similar protein/DNA sequences. 

The function has the following arguments: 

  * _seq1_ : First Protein/DNA sequence
  * _seq2_ : Second Protein/DNA sequence
  * _sub_matrix_ : Substitution matrix. E.g. BLOSUM50, BLOSUM62, BLOSUM80, and etc. 
  * _go_ : Gap opening penalty. Default value is -2
  * _ge_ : Gap extension penalty. Default value is -2
  * _align_type_: Type of alignment. 'gbl': global alignment and 'lcl': local alignment. Default alignment is global alignment.  

In [ ]:
def Aligntwo(seq1, seq2, sub_matrix = blosum_62_df, go = -2, ge = -2, align_type = 'gbl'):
    
    S, position_dir = get_S(seq1, seq2, go = go, ge = ge, sub_matrix = sub_matrix, align_type = align_type)
    
    path_dir = find_optim_path(S, position_dir, 
                               S.shape[0] - 1, 
                               S.shape[1] - 1, path_dir = [],
                               align_type = align_type)

    #print(S)
    #print(position_dir)
    #print(path_dir)

    print('Number of calculations: {}'.format(len(seq1)*len(seq2)*3))

    if align_type == 'gbl':
        #score = S[-1,-1]
        score = np.max([np.max(S[:,-1]),np.max(S[-1,:])])
        print('score: {}'.format(score))
        print('Results:')
        visualize_global_alignment(seq1, seq2, path_dir)
        
    elif align_type == 'lcl':
        score = np.max(S)
        print('score: {}'.format(score))
        print('Results:')
        visualize_local_alignment(seq1, seq2, S, position_dir)

## Global/Local Alignment Examples

Here are some of the results of alignment of protein/DNA sequences. Each Alignment results from Aligntwo function was compared to the EMBOSS function - a free open source software analysis package developed for the needs of the molecular biology and bioinformatics user community. The alignment results from EMBOSS are written as comments underneath each code. For certain argument conditions such as gap opening penalty -2, it was impossible to impose the same conditions in EMBOSS because they did not support the option. Thus EMBOSS alignment results are missing in some argument conditions. 

## Protein Sequence Alignment

### List of random protein sequences

In [ ]:
protein_seq1 = 'TSINQET'
protein_seq2 = 'SVETD'

protein_seq3 = 'SSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPV\
QLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQH\
LIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCM\
GGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENL'

protein_seq4 = 'SCAVPSTDDYAGKYGLQLDFQQNGTAKSVTCTYSPELNKLFCQLAKTCPL\
LVRVESPPPRGSILRATAVYKKSEHVAEVVKRCPHHERSVEPGEDAAPPS\
HLMRVEGNLQAYYMEDVNSGRHSVCVPYEGPQVGTECTTVLYNYMCNSSC\
MGGMNRRPILTIITLETPQGLLLGRRCFEVRVCACPGRDRRTEEDNY'

protein_seq5 = 'PQNKAISSSY'
protein_seq6 = 'IHMWYTSSGN'

### Condition No. 1

* Substitution_matrix: BLOSUM62
* Gap opening penalty: -5
* Gap extension penalty: -1
* Type of alignment: Global 

In [ ]:
Aligntwo(protein_seq1, protein_seq2, sub_matrix = blosum_62_df, go = -5, ge = -1, align_type = 'gbl')

#EMBOSS alignment results: 

#Score: 11.0

#=======================================

#EMBOSS_001         1 TSINQET-      7
#                      |:  || 
#EMBOSS_001         1 -SV--ETD      5

#=======================================

Number of calculations: 105
score: 11.0
Results:
TSINQET_
 |*  || 
_SV__ETD


In [ ]:
Aligntwo(protein_seq3, protein_seq4, sub_matrix = blosum_62_df, go = -5, ge = -1, align_type = 'gbl')

#EMBOSS alignment results: 

# Score: 725.0

#=======================================

#EMBOSS_001         1 SSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPV     50
#                     |.:|||...|.|.||.:|.|..:||||||||||||.|||:|||||||||:
#EMBOSS_001         1 SCAVPSTDDYAGKYGLQLDFQQNGTAKSVTCTYSPELNKLFCQLAKTCPL     50
#
#EMBOSS_001        51 QLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSD-SDGLAPPQ     99
#                     .:.|:|.||.|:.:||.|:||:|:|:.|||:|||||||..: .:..|||.
#EMBOSS_001        51 LVRVESPPPRGSILRATAVYKKSEHVAEVVKRCPHHERSVEPGEDAAPPS    100
#
#EMBOSS_001       100 HLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSC    149
#                     ||:||||||:..|::|.|:.||||.||||.|:||::|||:.|||||||||
#EMBOSS_001       101 HLMRVEGNLQAYYMEDVNSGRHSVCVPYEGPQVGTECTTVLYNYMCNSSC    150
#
#EMBOSS_001       150 MGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENL    196
#                     ||||||||||||||||...|.||||..|||||||||||||||||:|.
#EMBOSS_001       151 MGGMNRRPILTIITLETPQGLLLGRRCFEVRVCACPGRDRRTEEDNY    197

#=======================================

Number of calculations: 115836
score: 725.0
Results:
SSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPV
|**|||***|*|*||**|*|***||||||||||||*|||*|||||||||*
SCAVPSTDDYAGKYGLQLDFQQNGTAKSVTCTYSPELNKLFCQLAKTCPL
QLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSD_SDGLAPPQ
***|*|*||*|***||*|*||*|*|**|||*|||||||*** ****|||*
LVRVESPPPRGSILRATAVYKKSEHVAEVVKRCPHHERSVEPGEDAAPPS
HLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSC
||*||||||***|**|*|**||||*||||*|*||**|||**|||||||||
HLMRVEGNLQAYYMEDVNSGRHSVCVPYEGPQVGTECTTVLYNYMCNSSC
MGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENL
||||||||||||||||***|*||||**|||||||||||||||||*|*
MGGMNRRPILTIITLETPQGLLLGRRCFEVRVCACPGRDRRTEEDNY


In [ ]:
Aligntwo(protein_seq5, protein_seq6, sub_matrix = blosum_62_df, go = -5, ge = -1, align_type = 'gbl')

#score is the same but alignment is different in EMBOSS

#EMBOSS alignment results: 

#Score: 16.0

#=======================================

#EMBOSS_001         1 PQNKAISSSY-----     10
#                          |...|     
#EMBOSS_001         1 -----IHMWYTSSGN     10

#=======================================

Number of calculations: 300
score: 6.0
Results:
PQNKAI_____SSSY
     |     ||**
_____IHMWYTSSGN


### Condition No. 2

* Substitution_matrix: BLOSUM50
* Gap opening penalty: -2
* Gap extension penalty: -2
* Type of alignment: Local 

Comparison of alignment result between Aligntwo and EMBOSS is not possible because EMBOSS gap opening penalty options only support -1, -5, -10 etc. 

In [ ]:
Aligntwo(protein_seq1, protein_seq2, sub_matrix = blosum_50_df, go = -2, ge = -2, align_type = 'lcl') 

Number of calculations: 105
score: 16.0
Results:
SINQET
|*  ||
SV__ET


In [ ]:
Aligntwo(protein_seq3, protein_seq4, sub_matrix = blosum_50_df, go = -2, ge = -2, align_type = 'lcl')

Number of calculations: 115836
score: 970.0
Results:
SSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPV
|**|||***|*|*||**|*|***||||||||||||*|||*|||||||||*
SCAVPSTDDYAGKYGLQLDFQQNGTAKSVTCTYSPELNKLFCQLAKTCPL
QL_WVDSTPPP_GTRV_RAMAIYKQSQHMTEVVRRCPHHERCS_D_S_DG
 | *|*| ||| |* * ||*|*||*|*|**|||*||||||| | * * |*
_LVRVES_PPPRGS_ILRATAVYKKSEHVAEVVKRCPHHER_SVEPGEDA
LAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYM
 |||*||*||||||***|**|*|**||||*||||*|*||**|||**||||
_APPSHLMRVEGNLQAYYMEDVNSGRHSVCVPYEGPQVGTECTTVLYNYM
CNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEE
|||||||||||||||||||||***|*||||**|||||||||||||||||*
CNSSCMGGMNRRPILTIITLETPQGLLLGRRCFEVRVCACPGRDRRTEED
N
|
N


In [ ]:
Aligntwo(protein_seq5, protein_seq6, sub_matrix = blosum_50_df, go = -2, ge = -2, align_type = 'lcl')

Number of calculations: 300
score: 9.0
Results:
SS
||
SS


### Condition No. 3

* Substitution_matrix: BLOSUM80
* Gap opening penalty: -5
* Gap extension penalty: -1
* Type of alignment: Local 

In [ ]:
Aligntwo(protein_seq1, protein_seq2, sub_matrix = blosum_80_df, go = -5, ge = -1, align_type = 'lcl')

#EMBOSS alignment results: 

#Score: 21

#EMBOSS_001         2 SINQET      7
#                     |:  ||
#EMBOSS_001         1 SV--ET      4

Number of calculations: 105
score: 13.0
Results:
SINQET
|*  ||
SV__ET


In [ ]:
Aligntwo(protein_seq3, protein_seq4, sub_matrix = blosum_80_df, go = -5, ge = -1, align_type = 'lcl')

#EMBOSS alignment results: 

#Score: 1168.0

#EMBOSS_001         1 SSSVPSQKT--YQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTC     48
#                     |.:|||  |  |.|.||.:|.|.::||||||||||||.|||:||||||||
#EMBOSS_001         1 SCAVPS--TDDYAGKYGLQLDFQQNGTAKSVTCTYSPELNKLFCQLAKTC     48
#
#EMBOSS_001        49 P--VQLWVDSTPPP-GTRVRAMAIYKQSQHMTEVVRRCPHHERCS-----     90
#                     |  |:  |:| ||| |:.:||.|:||:|:|:.|||:||||||| |     
#EMBOSS_001        49 PLLVR--VES-PPPRGSILRATAVYKKSEHVAEVVKRCPHHER-SVEPGE     94
#
#EMBOSS_001        91 DSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIH    140
#                     |:   |||.||:||||||:..|::|.|:.||||.||||.|:||::|||:.
#EMBOSS_001        95 DA---APPSHLMRVEGNLQAYYMEDVNSGRHSVCVPYEGPQVGTECTTVL    141
#
#EMBOSS_001       141 YNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRR    190
#                     |||||||||||||||||||||||||...|.||||..||||||||||||||
#EMBOSS_001       142 YNYMCNSSCMGGMNRRPILTIITLETPQGLLLGRRCFEVRVCACPGRDRR    191
#
#EMBOSS_001       191 TEEEN    195
#                     |||:|
#EMBOSS_001       192 TEEDN    196


Number of calculations: 115836
score: 763.0
Results:
SSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPV
|**|||***|*|*||**|*|***||||||||||||*|||*|||||||||*
SCAVPSTDDYAGKYGLQLDFQQNGTAKSVTCTYSPELNKLFCQLAKTCPL
QLWVDSTPPP_GTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDG__LAP
***|*| ||| |***||*|*||*|*|**|||*||||||| |***|  *||
LVRVES_PPPRGSILRATAVYKKSEHVAEVVKRCPHHER_SVEPGEDAAP
PQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNS
|*||*||||||***|**|*|**||||*||||*|*||**|||**|||||||
PSHLMRVEGNLQAYYMEDVNSGRHSVCVPYEGPQVGTECTTVLYNYMCNS
SCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEEN
||||||||||||||||||***|*||||**|||||||||||||||||*|
SCMGGMNRRPILTIITLETPQGLLLGRRCFEVRVCACPGRDRRTEEDN


In [ ]:
Aligntwo(protein_seq5, protein_seq6, sub_matrix = blosum_80_df, go = -5, ge = -1, align_type = 'lcl')

#EMBOSS alignment results: 

#Score: 16.0

#EMBOSS_001         7 SSS      9
#                     :||
#EMBOSS_001         6 TSS      8

Number of calculations: 300
score: 8.0
Results:
SS
||
SS


## DNA Sequence Alignment

### List of Random DNA sequences

In [ ]:
dna_seq1 = 'CTAGGAT'
dna_seq2 = 'TACGT'

dna_seq3 = 'TAGGCAGAAA' 
dna_seq4 = 'CTGGACGTAC'

dna_seq5 = 'GATGTACGACCTGAGATCCT'  
dna_seq6 = 'CTGCTGATGGATCAGCGGTG'

### Condition No. 1

* Substitution matrix: dna_sub_matrix
* Gap opening penalty: -2
* Gap extension penalty: -2
* Alignment type: Global

In [ ]:
Aligntwo(dna_seq1, dna_seq2, sub_matrix = dna_sub_matrix, go = -2, ge = -2, align_type = 'gbl')

Number of calculations: 105
score: 1
Results:
CTAGGAT
 ||*| |
_TACG_T


In [ ]:
Aligntwo(dna_seq3, dna_seq4, sub_matrix = dna_sub_matrix, go = -2, ge = -2, align_type = 'gbl')

Number of calculations: 300
score: -1
Results:
_TAGGCAGAAA
 | ||**|*|*
CT_GGACGTAC


In [ ]:
Aligntwo(dna_seq5, dna_seq6, sub_matrix = dna_sub_matrix, go = -2, ge = -2, align_type = 'gbl')

Number of calculations: 1200
score: -4
Results:
__GATGTACGACCTGAGATCCT
  |*||***||*|*|*|*|  *
CTGCTGATGGATCAGCGGT__G


### Condition No. 2

* Substitution matrix: dna_sub_matrix
* Gap opening penalty: -5
* Gap extension penalty: -1
* Alignment type: Local

In [ ]:
Aligntwo(dna_seq1, dna_seq2, sub_matrix = dna_sub_matrix, go = -5, ge = -1, align_type = 'lcl')

# Score: 11.0
# 
#
#=======================================

#EMBOSS_001         2 TAGG      5
#                     ||.|
#EMBOSS_001         1 TACG      4


#---------------------------------------

Number of calculations: 105
score: 2
Results:
TAGG
||*|
TACG


In [ ]:
Aligntwo(dna_seq3, dna_seq4, sub_matrix = dna_sub_matrix, go = -5, ge = -1, align_type = 'lcl')

# Score: 11.0
# 
#
#=======================================

#EMBOSS_001         3 GGCA-GAA      9
#                     || | |.|
#EMBOSS_001         3 GG-ACGTA      9


#---------------------------------------
#---------------------------------------

Number of calculations: 300
score: 2
Results:
TA
||
TA


In [ ]:
Aligntwo(dna_seq5, dna_seq6, sub_matrix = dna_sub_matrix, go = -5, ge = -1, align_type = 'lcl')

# Score: 34.0
# 
#
#=======================================

#EMBOSS_001        11 CTGA--GATC     18
#                     ||||  ||||
#EMBOSS_001         4 CTGATGGATC     13


#---------------------------------------

Number of calculations: 1200
score: 4
Results:
GATGTA
||||*|
GATGGA


## Comparison of Aligntwo and EMBOSS Summary

To verify if the Aligntwo function works properly, I have compared the results of 3 pairs of protein sequences and 2 pairs DNA sequence alignments to EMBOSS sequence alignment results. 

* _Protein Alignment Condition #1_: 
  Alignment and score results were identical

* _Protein Alignment Condition #2_: Comparison of alignment result between Aligntwo and EMBOSS is not possible because EMBOSS gap opening penalty options only support -1, -5, -10 etc. 

* _Protein Alignment Condition #3_: Alignment was mostly identical except for protein_seq5 and protein_seq6. Alignment score results showed significant difference as well. I suspect that there is a difference in the BLOSUM80 substitution matrices that have been utilized. 

* _DNA Alignment Condition #1_: Comparison of alignment result between Aligntwo and EMBOSS is not possible because EMBOSS gap opening penalty options only support -1, -5, -10 etc. 

* _DNA Alignment Condition #2_: Aligntwo and EMBOSS outputed the same alignment for dna_seq1 and dna_seq2 only and all alignment scores showed a difference. I suspect that this is due to the difference in the DNA substitution matrix that have been utilized in both funcitons. In Aligntwo, a score of +1 was given for a match and a score of -1 for a mismatch regardless of the type of match or mismatch (e.g. A-G, C-G, T-A). Although the EMBOSS function does not explicitly describe how it scores each match and mismatch, I suspect that it gives different a score for a transition/transversion mutations. This would explain the difference in the alignment and the alignment score between the two functions

 



## Advantages and Limitations of the Aligntwo Function

After comparing the Aligntwo Function to EMBOSS, I discovered a few benefits and limitations the Aligntwo Function has in comparison to EBOSS. 

First, the EMBOSS function does not support a wide range of gap opening and extension penalty scores compared to Aligntwo. For instance, the EMBOSS function gap opening penalty options are -1, -5, -10, -20, -25, -50, -100. In comparison, the Aligntwo function allows any integer or float number to be entered by the user as a gap opening/extension penalty. 

However, the Aligntwo function lacks in many different aspects such as alignment visualization. The EMBOSS function visualizes the start index and the end index of the alignment, which is a feature Aligntwo does not yet support. 

Another limitation of the Aligntwo function is its inability to distinguish different types of substitution mutations. In EMBOSS, mismatch alignments are visualized as either '.' or ':' while Aligntwo function only visualizes mismatch alignments as '*'. 

In this regard, although Aligntwo confers greater versatility to its users, it still has alot of space for improvement. 